In [1]:
import subprocess
from pathlib import Path
from concurrent.futures import ProcessPoolExecutor

In [2]:
import sys
sys.path.append('../src')
from utils import run_cmd

In [3]:
program = '/home/chen1i6c04/time_filt.py'

In [4]:
dirpath = Path('/media/NAS/NanoporeOutput/20210721_clBC_Test50/1_Guppy')
sequencing_summary = dirpath/'sequencing_summary.txt'
outpath = Path('/media/NAS/Central_Lab_Storage/MinION/mNGS/20210721clBC_Beads/time_filt/fastq')
hours = (2, 4, 16)

In [5]:
cmds = []
for idx in (16, 17, 19, 20):
    barcode = f"barcode{idx:02}"
    barcode_dir = dirpath/barcode
    for hour in hours:
        outfile = outpath/(barcode + f'_{hour:02}h.fastq.gz')
        cmd = f"python {program} -s {sequencing_summary} -t {60*60*hour} {barcode_dir/'*.fastq'} | nanoq -l 1000 -q 10 | pigz -9 -p 2 > {outfile}"
        cmds.append(cmd)

In [6]:
with ProcessPoolExecutor(10) as executor:
    executor.map(run_cmd, cmds)